# Clasificación no supervisada con serie temporal NDVI de Terra/MODIS

In [3]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 83.0 MB/s eta 0:00:00


### Cargar librerías

In [4]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-my-francodbarr')

In [5]:
#importar librerías

import pandas as pd
import numpy as np
from datetime import datetime as dt
import geemap
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
from shapely.geometry import Point
import rasterio
import random

Funciones

In [6]:
def getCoordsSHP(gdfshpFile):
  '''
  This function takes a polygon vector layer file passed as a GeoDataFrame. Then
  it gets all the coordinates as a tuple of tuples.
  '''

  gdfiterf = gdfshpFile.iterfeatures()
  tupleFile = next(gdfiterf)['geometry']['coordinates']

  return tupleFile

### Seleccionar la colección y filtrarla

In [ ]:
#selecciona la colección MODIS MOD13Q1 NDVI
MODIS = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI') ##MOD13Q1.061 Terra Vegetation Indices 16-Day Global 250m

## establece el rango de fechas de interés
startDate = '2000-02-18'
endDate = '2022-12-31' ## la coleccion llega hasta 2023-02-02, aunque se informa que llega hasta julio 2023 (búsqueda agosto)

# filtrar por fechas
MODIS = MODIS.filterDate(startDate, endDate)

### Vector BBSS

In [8]:
!wget https://github.com/francobarrionuevoenv21/ClusteringArroceras_Incendios_SanJavier/blob/main/Primary_data/Planicie_con_paleocauces_NW-SW_4326.geojson

--2025-05-28 03:20:25--  https://github.com/francobarrionuevoenv21/ClusteringArroceras_Incendios_SanJavier/blob/main/Primary_data/Planicie_con_paleocauces_NW-SW_4326.geojson
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Planicie_con_paleocauces_NW-SW_4326.geojson’

Planicie_con_paleoc     [ <=>                ] 337.40K  --.-KB/s    in 0.03s   

2025-05-28 03:20:25 (10.1 MB/s) - ‘Planicie_con_paleocauces_NW-SW_4326.geojson’ saved [345501]



In [ ]:
targetCRS = "EPSG:4326"
vectorPPCC = gpd.read_file('/content/Planicie_con_paleocauces_NW-SW_4326.geojson').to_crs(targetCRS)

In [ ]:
# El siguiente codigo recupera todas las coordenadas de un vector, y a partir de ellas
# crea una lista de tuplas. Con esta lista se genera finalmente un poligono.

tupleOfTuples01 = getCoordsSHP(vectorPPCC)

# Converting coordinates into a list of lists
listOfLists01 = [list(t[:2]) for t in tupleOfTuples01[0]]

# Coordinates to a Polygon
polygon01 = ee.Geometry.Polygon(listOfLists01)

# Polygon to a EE Feature Collection (For plotting the vector of the region)
featureCollection01 = ee.FeatureCollection(polygon01)

In [ ]:
# ver tamaño de la colección - es una colección con 528 elementos en mi área de prueba - 526 si dejo hasta diciembre 2022
MODIS

In [ ]:
# Define a function to unmask the NDVI band
def unmaskNDVI(image):
    return image.select('NDVI').unmask(-2000)

# Map the unmask function over the entire collection
MODIS_unmasked = MODIS.map(unmaskNDVI)

In [ ]:
# Convertir la colección a un stack - una imagen de NDVI por fecha
MODIS_stack = MODIS_unmasked.toBands()

MODIS_stack # ya no es una colección, es una imagen con 526 bandas en mi área de prueba

In [ ]:
#cortar con un shapefile
MODIS_stack = MODIS_stack.clipToCollection(featureCollection01)

### K-means

In [ ]:
# El kmeans de GEE necesita entrenar al menos con puntos al azar
# Te pide que le des puntos al azar para inicializar el algoritmo
# NO DAR muestras de entrenamiento

# Fuente: https://developers.google.com/earth-engine/guides/clustering

#Define a region in which to generate a sample of the input.

#primero defino las variables que piden más abajo
region = featureCollection01
scale = 250 ## este es el tamaño de pixel de MODIS en esta colección
#scale = 231.65635826395825 ## este es el tamaño de pixel de MODIS en esta colección

numPixels = 5000 # puntos de inicializacion --> no son de entrenamiento

# Make the training dataset, except for -999 (mascara)
#MODIS_randomtraining = MODIS_stack.sample(region = region, scale = scale, numPixels = numPixels)

MODIS_randomtraining = MODIS_stack.sample(region=region, scale=scale, numPixels=numPixels)

In [ ]:
# Iniciar una clasificación kmeans con N clusters

num_clusters = 50 # numero de clases
# empezar > 8 ==> David ya clasifico 8, incluyendo arroceras
kmeans = ee.Clusterer.wekaKMeans(num_clusters).train(MODIS_randomtraining)

In [ ]:
# Aplica la clasificación a la imagen
MODIS_kmeans = MODIS_stack.cluster(kmeans) # cada pixel va de 0-n_clusters-1

MODIS_kmeans = MODIS_kmeans.add(1) #sumar 1 para que no empiece en 0 el cluster numbering
# cada pixel va de 1-n_clusters

MODIS_kmeans = MODIS_kmeans.clip(featureCollection01)

In [ ]:
MODIS_kmeans # es una imagen de una banda, lo que está bien. espero que sea una clasificación.

In [ ]:
# generar mapa

# ver clasificación - en Python

Map = geemap.Map(center=(-32.06, -60.62), zoom=9)
Map.addLayer(MODIS_kmeans)

Map

Map(center=[-32.06, -60.62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [ ]:
#https://worldbank.github.io/OpenNightLights/tutorials/mod3_7_import_export_data.html

task = ee.batch.Export.image.toDrive(
        image = MODIS_kmeans.toDouble(),
        scale=scale,
        region = featureCollection01.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
        description = 'MODIS_MOD13Q1_NDVI_kmeans50-17-04.tif',
        crs = 'EPSG:4326',
        maxPixels = 1e10,
        fileFormat = "GeoTIFF",
        folder ='earthengine'
  )

task.start()

## Resumen de las clases

#### Extraccion de los datos para su visualizacion en forma de SSTT de NDVI

In [ ]:
# https://drive.google.com/file/d/1yRZ7BXNJlEytsreXhU2A4wI_MQkPnlMt/view?usp=sharing

!gdown --id 1yRZ7BXNJlEytsreXhU2A4wI_MQkPnlMt

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1yRZ7BXNJlEytsreXhU2A4wI_MQkPnlMt
To: /content/MODIS_MOD13Q1_NDVI_kmeans50-17-04.tif.tif
100% 60.5k/60.5k [00:00<00:00, 87.8MB/s]


In [ ]:
# Load raster classification
raster_file = "/content/MODIS_MOD13Q1_NDVI_kmeans50-17-04.tif.tif"
with rasterio.open(raster_file) as src:
    classification = src.read(1)
    transform = src.transform
    num_clusters = len(np.unique(classification))

# Create vector layer with random points
num_points_per_cluster = 100 #
#num_points_per_cluster = 5 #prueba con pocos puntos

points = []
cluster_numbers = []

for cluster_id in range(1, num_clusters + 1):
    cluster_indices = np.where(classification == cluster_id)
    sample_indices = random.sample(range(len(cluster_indices[0])), min(num_points_per_cluster, len(cluster_indices[0])))

    for index in sample_indices:
        row, col = cluster_indices[0][index], cluster_indices[1][index]
        x, y = rasterio.transform.xy(transform, row, col)
        points.append(Point(x, y))
        cluster_numbers.append(cluster_id)


# Create a GeoDataFrame from the points
data = {'geometry': points, 'cluster': cluster_numbers}
gdf = gpd.GeoDataFrame(data, crs=src.crs)

# Save the GeoDataFrame to a shapefile
output_shapefile = "kmeans_random_points.shp"
gdf.to_file(output_shapefile)

print("Random points saved to:", output_shapefile)

Random points saved to: kmeans_random_points.shp


In [ ]:
modis_collection = MODIS
#modis_collection = MODIS.filterDate("2022-01-01", "2022-01-31") #prueba con pocas imagenes


clusters = gdf["cluster"].unique()

# Create a function to extract NDVI values for a point
def get_ndvi_for_point(feature):
    point = feature.geometry()
    ndvi_values = modis_collection.getRegion(point, scale=250).getInfo()
    return ndvi_values

# Loop through each cluster and extract NDVI values for each point
ndvi_data = []

for cluster in clusters:
    cluster_gdf = gdf[gdf["cluster"] == cluster]
    for index, row in cluster_gdf.iterrows():
        point = Point(row["geometry"].x, row["geometry"].y)
        feature = ee.Feature(ee.Geometry.Point(point.x, point.y))
        ndvi_values = get_ndvi_for_point(feature)

        # Extract NDVI values for each scene
        for scene_values in ndvi_values[1:]:
            timestamp_milliseconds = int(scene_values[3])
            timestamp_seconds = timestamp_milliseconds / 1000
            dt_object = dt.utcfromtimestamp(timestamp_seconds)
            formatted_date = dt_object.strftime('%Y-%m-%d')

            scene_ndvi_values = scene_values[4:]
            ndvi_data.append([cluster, point.x, point.y, formatted_date, *scene_ndvi_values])

# Create a dataframe from the extracted data
columns = ["cluster", "longitude", "latitude", "date"] + ["NDVI" + str(i) for i in range(1, len(ndvi_data[0]) - 3)]
ndvi_df = pd.DataFrame(ndvi_data, columns=columns)

# NOTA: este paso para el Delta tarda unas 2 horas

In [ ]:
#llenar los NoData con -2000 (igual que hicimos en la imagen con unmask, ahora lo hace en la tabla)
ndvi_df = ndvi_df.fillna(-2000)

In [ ]:
# Display the dataFrame
ndvi_df

,cluster,longitude,latitude,date,NDVI1
0,1,-60.051254,-30.651640,2000-02-18,7395.0
1,1,-60.051254,-30.651640,2000-03-05,7517.0
2,1,-60.051254,-30.651640,2000-03-21,7731.0
3,1,-60.051254,-30.651640,2000-04-06,8263.0
4,1,-60.051254,-30.651640,2000-04-22,8256.0
...,...,...,...,...,...
2543205,50,-60.190493,-30.685327,2022-10-16,3028.0
2543206,50,-60.190493,-30.685327,2022-11-01,3042.0
2543207,50,-60.190493,-30.685327,2022-11-17,4194.0
2543208,50,-60.190493,-30.685327,2022-12-03,7065.0


In [ ]:
# Pivot the DataFrame to have one column per date
pivot_df = ndvi_df.pivot_table(
    index=["cluster", "longitude", "latitude"],
    columns="date",
    values=["NDVI1"],  # List all the NDVI columns here
    aggfunc="first"  # You can adjust the aggregation function if needed
)

# Reset the index to make columns regular columns
pivot_df.reset_index(inplace=True)

# Display the pivoted DataFrame
pivot_df

cluster  longitude   latitude      NDVI1                        \
date                               2000-02-18 2000-03-05 2000-03-21   
0          1 -60.152314 -30.651640     7037.0     7080.0     6628.0   
1          1 -60.152314 -30.642657     7104.0     6867.0     5120.0   
2          1 -60.152314 -30.635920     6332.0     6519.0     5008.0   
3          1 -60.150069 -30.647149     7408.0     7166.0     5640.0   
4          1 -60.147823 -30.647149     7384.0     7284.0     5851.0   
...      ...        ...        ...        ...        ...        ...   
4830      50 -59.983880 -30.375408     7069.0     5389.0     7055.0   
4831      50 -59.983880 -30.373163     7173.0     5571.0     6894.0   
4832      50 -59.977143 -30.375408     5884.0     4285.0     6043.0   
4833      50 -59.968160 -30.375408     4480.0     3857.0     5408.0   
4834      50 -59.965914 -30.373163     6444.0     5193.0     5599.0   

                                                  ...                        \
date 2000-04-06 2000-04-22 2000-05-08 2000-05-24  ... 2022-07-28 2022-08-13   
0        7767.0     7267.0     7548.0     6825.0  ...     5395.0     5549.0   
1        6631.0     6690.0     6722.0     6649.0  ...     5589.0     6056.0   
2        7757.0     6428.0     6796.0     6221.0  ...     5380.0     5208.0   
3        7737.0     7150.0     7491.0     6813.0  ...     5675.0     6173.0   
4        7737.0     7150.0     7491.0     7170.0  ...     5675.0     6173.0   
...         ...        ...        ...        ...  ...        ...        ...   
4830     7582.0     8263.0     8103.0     5778.0  ...     4483.0     3420.0   
4831     7582.0     8023.0     7849.0     5464.0  ...     3237.0     2729.0   
4832     7582.0     7333.0     7015.0     6424.0  ...     5972.0     3928.0   
4833     6646.0     7199.0     7509.0     6101.0  ...     3412.0     3003.0   
4834     7176.0     7364.0     6686.0     6273.0  ...     3927.0     3379.0   

                                                                        \
date 2022-08-29 2022-09-14 2022-09-30 2022-10-16 2022-11-01 2022-11-17   
0        5755.0     5548.0     4566.0     4422.0     4312.0     3869.0   
1        5580.0     5373.0     4763.0     4292.0     4366.0     4245.0   
2        5363.0     4623.0     4304.0     4075.0     4239.0     4105.0   
3        6132.0     5661.0     4632.0     4338.0     4116.0     3931.0   
4        6264.0     5804.0     4586.0     4363.0     4059.0     4200.0   
...         ...        ...        ...        ...        ...        ...   
4830     3965.0     4526.0     4697.0     3504.0     4293.0     3886.0   
4831     3279.0     3409.0     2716.0     2795.0     2805.0     2698.0   
4832     5671.0     5294.0     4697.0     4171.0     5245.0     7010.0   
4833     3148.0     3021.0     2979.0     3695.0     4854.0     6301.0   
4834     3587.0     3485.0     4440.0     3907.0     4734.0     5505.0   

                            
date 2022-12-03 2022-12-19  
0        4610.0     4772.0  
1        4476.0     4380.0  
2        4526.0     4504.0  
3        4502.0     4156.0  
4        4502.0     4444.0  
...         ...        ...  
4830     5154.0     5698.0  
4831     3649.0     6980.0  
4832     8169.0     7535.0  
4833     8813.0     8702.0  
4834     8750.0     8456.0  

[4835 rows x 529 columns]

In [ ]:
#export
pivot_df.to_csv("./kmeans_ndvi_pivot.csv", index=False)

ndvi_df.to_csv("./kmeans_ndvi.csv", index=False)